In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *

# 从头开始构建语言模型

现在我们准备深入……深入深度学习！你已经学会了如何训练一个基本的神经网络，但是如何从这里开始创建最先进的模型呢？在本书的这一部分，我们将揭开所有神秘的面纱，从语言模型开始。

你在<<chapter_nlp>>中看到了如何微调预训练的语言模型来构建文本分类器。在本章中，我们将向你解释模型内部究竟是什么，以及RNN（循环神经网络）是什么。首先，让我们收集一些数据，以便我们可以快速原型化我们的各种模型。

## 数据

每当我们开始研究一个新问题时，我们总是首先尝试想到最简单的数据集，这样我们就可以快速、容易地尝试方法并解释结果。几年前我们开始研究语言建模时，并没有发现任何可以快速原型化的数据集，所以我们自己创建了一个。我们称之为*Human Numbers*，它简单地包含了用英语写出的前10,000个数字。

> j: 即使在经验丰富的从业者中，我经常看到的一种常见实际错误是在分析过程中未能在适当的时候使用适当的数据集。特别是，大多数人倾向于从太大和太复杂的数据集开始。

我们可以以通常的方式下载、解压并查看我们的数据集：

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

In [ ]:
#hide
Path.BASE_PATH = path

In [ ]:
path.ls()

(#2) [Path('train.txt'),Path('valid.txt')]

让我们打开这两个文件，看看里面有什么。起初，我们将所有文本连接在一起，并忽略数据集给出的训练/验证分割（我们稍后会回到这个问题）：

In [ ]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

我们取所有这些行，并将它们连接成一个大的流。为了标记从一个数字到下一个数字的转换，我们使用`.`作为分隔符：

In [ ]:
text = ' . '.join([l.strip() for l in lines])
text[:100]

'one . two . three . four . five . six . seven . eight . nine . ten . eleven . twelve . thirteen . fo'

我们可以通过在空格处分割来对这个数据集进行分词：

In [ ]:
tokens = text.split(' ')
tokens[:10]

['one', '.', 'two', '.', 'three', '.', 'four', '.', 'five', '.']

为了进行数值化，我们必须创建一个包含所有独特标记（我们的*词汇表*）的列表：

In [ ]:
vocab = L(*tokens).unique()
vocab

(#30) ['one','.','two','three','four','five','six','seven','eight','nine'...]

然后我们可以通过在词汇表中查找每个标记的索引，将我们的标记转换为数字：

In [ ]:
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
nums

(#63095) [0,1,2,1,3,1,4,1,5,1...]

现在我们有了一个小型数据集，在这个数据集上进行语言建模应该是一个简单的任务，我们可以构建我们的第一个模型。

## 我们从头开始的第一个语言模型

将这个转换为神经网络的一个简单方法是指定我们将基于前三个词来预测每个词。我们可以创建一个包含每个三个词序列的列表作为我们的独立变量，以及每个序列之后的一个词作为因变量。

我们可以用纯Python来做这件事。首先，让我们用标记来确认它的样子：

In [ ]:
L((tokens[i:i+3], tokens[i+3]) for i in range(0,len(tokens)-4,3))

(#21031) [(['one', '.', 'two'], '.'),(['.', 'three', '.'], 'four'),(['four', '.', 'five'], '.'),(['.', 'six', '.'], 'seven'),(['seven', '.', 'eight'], '.'),(['.', 'nine', '.'], 'ten'),(['ten', '.', 'eleven'], '.'),(['.', 'twelve', '.'], 'thirteen'),(['thirteen', '.', 'fourteen'], '.'),(['.', 'fifteen', '.'], 'sixteen')...]

现在我们将使用数值化的张量来做这件事，这实际上是模型将要使用的：

In [ ]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

(#21031) [(tensor([0, 1, 2]), 1),(tensor([1, 3, 1]), 4),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 7),(tensor([7, 1, 8]), 1),(tensor([1, 9, 1]), 10),(tensor([10,  1, 11]), 1),(tensor([ 1, 12,  1]), 13),(tensor([13,  1, 14]), 1),(tensor([ 1, 15,  1]), 16)...]

我们可以很容易地使用`DataLoader`类来批量处理这些数据。现在我们将随机分割序列：

In [ ]:
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

现在我们可以创建一个神经网络架构，它接受三个词作为输入，并返回词汇表中每个可能的下一个词的概率预测。我们将使用三个标准的线性层，但有两个调整。

第一个调整是，第一个线性层将仅使用第一个词的嵌入作为激活值，第二层将使用第二个词的嵌入加上第一层的输出激活值，第三层将使用第三个词的嵌入加上第二层的输出激活值。这样做的关键效果是，每个词都是在它之前任何词的信息上下文中被解释的。

第二个调整是，这三个层将使用相同的权重矩阵。一个词对前词激活值的影响不应该根据词的位置而改变。换句话说，随着数据在层之间传递，激活值会发生变化，但层权重本身不会从一层改变到另一层。因此，一个层不会学习一个序列位置；它必须学会处理所有位置。

由于层权重不改变，你可能会将顺序层视为重复的“相同层”。实际上，PyTorch使这一点具体化；我们可以只创建一个层，并多次使用它。

### 我们在PyTorch中的语言模型

现在我们可以创建我们之前描述的语言模型模块：

In [ ]:
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

正如你看到的，我们创建了三个层：

- 嵌入层（`i_h`，表示*输入*到*隐藏*）
- 创建下一个词激活值的线性层（`h_h`，表示*隐藏*到*隐藏*）
- 预测第四个词的最终线性层（`h_o`，表示*隐藏*到*输出*）

这可能用图形表示更容易理解，所以让我们定义一个基本神经网络的简单图形表示。<<img_simple_nn>>展示了我们如何表示一个具有一个隐藏层的神经网络。

<img alt="Pictorial representation of simple neural network" width="400" src="images/att_00020.png" caption="Pictorial representation of a simple neural network" id="img_simple_nn">

每个形状代表激活值：矩形代表输入，圆形代表隐藏（内部）层的激活值，三角形代表输出激活值。我们将在本章的所有图表中使用这些形状（在<<img_shapes>>中总结）。

<img alt="Shapes used in our pictorial representations" width="200" src="images/att_00021.png" id="img_shapes" caption="Shapes used in our pictorial representations">

箭头代表实际的层计算——即线性层后跟激活函数。使用这种表示法，<<LM_rep>>展示了我们简单的语言模型的样子。

<img alt="Representation of our basic language model" width="500" caption="Representation of our basic language model" id="lm_rep" src="images/att_00022.png">

为了简化，我们已经从每个箭头中移除了层计算的细节。我们还对箭头进行了颜色编码，使得所有具有相同颜色的箭头具有相同的权重矩阵。例如，所有的输入层都使用相同的嵌入矩阵，所以它们都有相同的颜色（绿色）。

让我们尝试训练这个模型，看看效果如何：

In [ ]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.824297,1.970941,0.467554,00:02
1,1.386973,1.823242,0.467554,00:02
2,1.417556,1.654497,0.494414,00:02
3,1.376440,1.650849,0.494414,00:02


为了看看这是否有用，让我们检查一个非常简单的模型会给我们什么结果。在这种情况下，我们总是可以预测最常见的标记，所以让我们找出在我们的验证集中哪个标记最常是目标：

In [ ]:
n,counts = 0,torch.zeros(len(vocab))
for x,y in dls.valid:
    n += y.shape[0]
    for i in range_of(vocab): counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)
idx, vocab[idx.item()], counts[idx].item()/n

(tensor(29), 'thousand', 0.15165200855716662)

最常见的标记索引是29，对应于标记`thousand`。总是预测这个标记将给我们大约15%的准确率，所以我们的表现要好得多！

> A: 我最初的猜测是分隔符会是最常见的标记，因为每个数字都有一个。但是查看`tokens`提醒我，大数字是用很多单词写的，所以在达到10,000的路上你会写很多次“thousand”：五千，五千零一，五千零二，等等。哎呀！查看你的数据对于注意到微妙的特征以及非常明显的特征都很棒。

这是一个不错的初步基线。让我们看看如何用循环来重构它。

### 我们的第一个循环神经网络

查看我们模块的代码，我们可以通过用`for`循环替换调用层的重复代码来简化它。除了使我们的代码更简单之外，这还有一个好处，就是我们将能够同样好地将我们的模块应用于不同长度的标记序列——我们不会仅限于长度为三的标记列表：

In [ ]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

让我们检查一下使用这种重构方法是否能得到相同的结果：

In [ ]:
learn = Learner(dls, LMModel2(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.816274,1.964143,0.460185,00:02
1,1.423805,1.739964,0.473259,00:02
2,1.430327,1.685172,0.485382,00:02
3,1.388390,1.657033,0.470406,00:02


我们也可以以完全相同的方式重构我们的图形表示，如<<img_basic_rnn>>所示（我们在这里也移除了激活大小的细节，并使用了与<<img_lm_rep>>中相同的箭头颜色）。

<img alt="Basic recurrent neural network" width="400" caption="Basic recurrent neural network" id="basic_rnn" src="images/att_00070.png">

你会看到有一组激活值在每次循环时都在更新，存储在变量`h`中——这被称为*隐藏状态*。

> 术语：隐藏状态：在循环神经网络的每一步中更新的激活值。

一个像这样使用循环定义的神经网络被称为*循环神经网络*（RNN）。重要的是要意识到，RNN并不是一种复杂的新架构，而只是使用`for`循环重构的多层神经网络。

> A: 我的真正看法：如果它们被称为“循环神经网络”或者LNNs，它们看起来就不会那么令人生畏了，至少减少了50%！

现在我们知道了RNN是什么，让我们尝试让它变得更好一些。

## 改进循环神经网络（RNN）

观察我们的RNN代码，一个问题是我们为每个新的输入序列初始化隐藏状态为零。为什么这是一个问题？我们让样本序列变短，以便它们可以轻松地放入批次中。但是，如果我们正确地对样本进行排序，这些样本序列将按顺序被模型读取，暴露出原始序列的长段。

我们还可以关注的另一件事是增加信号：为什么只预测第四个词，当我们可以使用中间预测来预测第二个和第三个词呢？

让我们看看如何实现这些变化，从添加一些状态开始。

### 保持 RNN 状态

因为我们为每个新样本将模型的隐藏状态初始化为零，我们实际上丢弃了我们到目前为止看到的句子的所有信息，这意味着我们的模型实际上不知道我们在整体计数序列中的位置。这个问题很容易解决；我们可以简单地将隐藏状态的初始化移动到`__init__`方法中。

但这个修复将产生自己的微妙但重要的问题。它实际上使我们的神经网络的深度与我们文档中的整个标记数量一样。例如，如果我们的数据集中有10,000个标记，我们将创建一个10,000层的神经网络。

为了理解为什么会这样，考虑我们在使用`for`循环重构之前的循环神经网络的原始图形表示（见<<img_lm_rep>>）。你可以看到每一层都对应一个标记输入。当我们谈论在重构`for`循环之前的循环神经网络的表示时，我们称之为*展开表示*。在试图理解RNN时，考虑展开表示通常是有帮助的。

一个10,000层神经网络的问题是，如果你和当你到达数据集中的第10,000个词，你仍然需要计算所有回到第一层的导数。这将非常慢，并且非常耗费内存。你很可能无法在你的GPU上存储一个迷你批次。

解决这个问题的方法是告诉PyTorch我们不想通过整个隐含的神经网络反向传播导数。相反，我们只会保留最后三层的梯度。要在PyTorch中移除所有梯度历史，我们使用`detach`方法。

这是我们RNN的新版本。现在它是有状态的，因为它在不同调用`forward`之间记住它的激活值，这代表了它在批次中不同样本的使用：

In [ ]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    
    def reset(self): self.h = 0

这个模型将具有相同的激活值，无论我们选择什么序列长度，因为隐藏状态会记住前一个批次中的最后激活值。唯一不同的是在每个步骤计算的梯度：它们只会在序列长度标记的过去计算，而不是整个流。这种方法称为*通过时间的反向传播*（BPTT）。

> 术语：通过时间的反向传播（BPTT）：将一个实际上每个时间步只有一个层的神经网络（通常使用循环重构）视为一个大模型，并以通常的方式在其上计算梯度。为了避免耗尽内存和时间，我们通常使用_截断的_ BPTT，它在隐藏状态中每隔几个时间步“分离”计算步骤的历史。

为了使用`LMModel3`，我们需要确保样本将以特定的顺序被看到。正如我们在<<chapter_nlp>>中看到的，如果第一个批次的第一行是我们的`dset[0]`，那么第二个批次应该有`dset[1]`作为第一行，这样模型就能看到文本的流动。

`LMDataLoader`在<<chapter_nlp>>中为我们做了这件事。这次我们将自己来做。

为了实现这一点，我们将重新排列我们的数据集。首先，我们将样本分成`m = len(dset) // bs`组（这相当于将整个连接的数据集分割成64个同样大小的部分，因为我们在这里使用`bs=64`）。`m`是每个部分的长度。例如，如果我们使用整个数据集（尽管我们很快就会将其分割为训练和验证部分），那将是：

In [ ]:
m = len(seqs)//bs
m,bs,len(seqs)

(328, 64, 21031)

第一个批次将由以下样本组成：

    (0, m, 2*m, ..., (bs-1)*m)

第二个批次的样本：

    (1, m+1, 2*m+1, ..., (bs-1)*m+1)

以此类推。这样，在每个时代（epoch），模型将在批次的每一行看到大小为`3*m`的连续文本块（因为每个文本的大小为3）。

以下函数执行这个重新索引：

In [ ]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

然后我们在构建`DataLoaders`时只需传递`drop_last=True`来丢弃最后一个形状不是`bs`的批次。我们还传递`shuffle=False`以确保文本按顺序读取：

In [ ]:
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

我们添加的最后一件事是通过一个`Callback`对训练循环进行微调。我们将在<<chapter_accel_sgd>>中更多地讨论回调。这个回调将在每个时代的开始和每个验证阶段之前调用我们模型的`reset`方法。由于我们实现了这个方法来将模型的隐藏状态置零，这将确保我们在阅读这些连续的文本块之前以一个干净的州开始。我们也可以开始稍长一点的训练：

In [ ]:
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.677074,1.827367,0.467548,00:02
1,1.282722,1.870913,0.388942,00:02
2,1.090705,1.651793,0.462500,00:02
3,1.005092,1.613794,0.516587,00:02
4,0.965975,1.560775,0.551202,00:02
5,0.916182,1.595857,0.560577,00:02
6,0.897657,1.539733,0.574279,00:02
7,0.836274,1.585141,0.583173,00:02
8,0.805877,1.629808,0.586779,00:02
9,0.795096,1.651267,0.588942,00:02


这已经更好了！下一步是使用更多的目标并与中间预测进行比较。

### 创建更多信号

我们当前方法的另一个问题是，我们只为每三个输入词预测一个输出词。这意味着我们用来更新权重的反馈信号量没有达到最大。如果我们在每个单独的词之后预测下一个词，而不是每三个词之后，情况会更好，如<<stateful_rep>>所示。

<img alt="RNN predicting after every token" width="400" caption="RNN predicting after every token" id="stateful_rep" src="images/att_00024.png">

这很容易添加。我们首先需要改变我们的数据，使得因变量在每个三个输入词之后都有接下来的三个词。我们不是使用`3`，而是使用一个属性`sl`（表示序列长度），并使其稍大一些：

In [ ]:
sl = 16
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

查看`seqs`的第一个元素，我们可以看到它包含两个相同大小的列表。第二个列表与第一个列表相同，但偏移了一个元素：

In [ ]:
[L(vocab[o] for o in s) for s in seqs[0]]

[(#16) ['one','.','two','.','three','.','four','.','five','.'...],
 (#16) ['.','two','.','three','.','four','.','five','.','six'...]]

现在我们需要修改我们的模型，使其在每个词之后输出一个预测，而不仅仅是在三个词序列的末尾：

In [ ]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

这个模型将返回形状为`bs x sl x vocab_sz`的输出（因为我们在`dim=1`上进行了堆叠）。我们的目标形状为`bs x sl`，所以我们需要在`F.cross_entropy`中使用它们之前将它们展平：

In [ ]:
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

现在我们可以使用这个损失函数来训练模型：

In [ ]:
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.103298,2.874341,0.212565,00:01
1,2.231964,1.971280,0.462158,00:01
2,1.711358,1.813547,0.461182,00:01
3,1.448516,1.828176,0.483236,00:01
4,1.288630,1.659564,0.520671,00:01
5,1.161470,1.714023,0.554932,00:01
6,1.055568,1.660916,0.575033,00:01
7,0.960765,1.719624,0.591064,00:01
8,0.870153,1.839560,0.614665,00:01
9,0.808545,1.770278,0.624349,00:01


我们需要训练更长的时间，因为任务现在已经有所变化，变得更加复杂。但我们最终得到了一个好的结果……至少，有时候是这样。如果你多次运行它，你会看到在不同的运行中可以得到相当不同的结果。这是因为我们实际上有一个非常深的网络，这可能导致非常大或非常小的梯度。我们将在本章的下一部分看到如何处理这个问题。

现在，显然的改进模型的方法是加深网络：我们的RNN中只有一层线性层在隐藏状态和输出激活之间，所以也许我们可以通过增加更多的层来获得更好的结果。

## 多层循环神经网络（MLRNNs）

在多层循环神经网络（MLRNN）中，我们将激活值从我们的循环神经网络传递到第二个循环神经网络，如<<stacked_rnn_rep>>所示。

<img alt="2-layer RNN" width="550" caption="2-layer RNN" id="stacked_rnn_rep" src="images/att_00025.png">

展开表示如图<<unrolled_stack_rep>>所示（与<<lm_rep>>类似）。

<img alt="2-layer unrolled RNN" width="500" caption="Two-layer unrolled RNN" id="unrolled_stack_rep" src="images/att_00026.png">

让我们看看如何在实践中实现这个。

### 模型

我们可以通过使用PyTorch的`RNN`类来节省一些时间，它实现了我们之前创建的内容，并且还提供了堆叠多个RNN的选项，正如我们所讨论的那样：

In [ ]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [ ]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.055853,2.591640,0.437907,00:01
1,2.162359,1.787310,0.471598,00:01
2,1.710663,1.941807,0.321777,00:01
3,1.520783,1.999726,0.312012,00:01
4,1.330846,2.012902,0.413249,00:01
5,1.163297,1.896192,0.450684,00:01
6,1.033813,2.005209,0.434814,00:01
7,0.919090,2.047083,0.456706,00:01
8,0.822939,2.068031,0.468831,00:01
9,0.750180,2.136064,0.475098,00:01


现在这令人失望……我们之前的单层RNN表现得更好。为什么呢？原因是我们有一个更深的模型，导致激活值爆炸或消失。

### 激活值爆炸或消失

在实践中，从这种RNN创建准确的模型是困难的。如果我们不那么频繁地调用`detach`，并且有更多的层，我们将获得更好的结果——这给我们的RNN更长的时间范围来学习，并创建更丰富的特征。但这也意味着我们有一个更深的模型需要训练。深度学习发展的关键挑战之一就是如何训练这类模型。

这个挑战之所以存在，是因为当你多次乘以一个矩阵时会发生的情况。想想当你多次乘以一个数字时会发生什么。例如，如果你从1开始乘以2，你会得到序列1, 2, 4, 8, ...在32步之后，你已经到了4,294,967,296。如果你乘以0.5，你会得到0.5, 0.25, 0.125...在32步之后，它变成了0.00000000023。正如你看到的，即使稍微大于或小于1的数字，经过几次重复乘法后，也会导致我们的起始数字爆炸或消失。

因为矩阵乘法只是乘以数字然后加起来，所以完全相同的情况也会发生在重复的矩阵乘法中。而深度神经网络就是这样——每增加一层就是另一个矩阵乘法。这意味着深度神经网络很容易最终得到非常大或非常小的数字。

这是一个问题，因为计算机存储数字的方式（称为“浮点数”）意味着数字离零越远，它们的准确性就越低。在<<float_prec>>中的文章["What You Never Wanted to Know About Floating Point but Will Be Forced to Find Out"](http://www.volkerschatz.com/science/float.html)的图表显示了浮点数的精度如何随着数字线的变化而变化。

<img alt="Precision of floating point numbers" width="1000" caption="Precision of floating-point numbers" id="float_prec" src="images/fltscale.svg">

这种不准确性意味着在深度网络中，用于更新权重的梯度经常最终变为零或无穷大。这通常被称为*梯度消失*或*梯度爆炸*问题。这意味着在随机梯度下降（SGD）中，权重要么根本不更新，要么跳转到无穷大。无论哪种情况，它们都不会随着训练而改善。

研究人员已经开发了许多解决这个问题的方法，我们将在本书后面的章节中讨论。一个选择是以一种减少激活值爆炸可能性的方式改变层的定义。我们将在<<chapter_convolutions>>中讨论批量归一化时，以及在<<chapter_resnet>>中讨论ResNets时，看看这是如何实现的，尽管这些细节在实践中通常不重要（除非你是正在创造解决这个问题的新方法的研究人员）。另一个应对策略是通过仔细的初始化来处理，这将在<<chapter_foundations>>中探讨。

对于RNN，有两种类型的层经常被用来避免激活值爆炸：*门控循环单元*（GRUs）和*长短期记忆*（LSTM）层。这两种层在PyTorch中都有，并且可以作为RNN层的直接替代品。本书中我们只涵盖LSTM；有许多优秀的在线教程解释了GRUs，它们是LSTM设计的一个小变种。

## 长短时记忆

LSTM是一种由Jürgen Schmidhuber和Sepp Hochreiter在1997年引入的架构。在这种架构中，不是有一个，而是有两个隐藏状态。在我们的基础RNN中，隐藏状态是前一个时间步RNN的输出。然后这个隐藏状态负责两件事：

- 为输出层提供正确的信息，以预测正确的下一个标记
- 保留句子中发生的一切记忆

例如，考虑句子 "Henry has a dog and he likes his dog very much" 和 "Sophie has a dog and she likes her dog very much." 很明显，RNN需要记住句子开头的名字，才能预测*he/she*或*his/her*。

在实践中，RNN在保留句子早期发生的事情的记忆方面表现得非常糟糕，这就是在LSTM中引入另一个隐藏状态（称为*细胞状态*）的动机。细胞状态将负责保持*长短期记忆*，而隐藏状态将专注于预测下一个标记。让我们仔细看看这是如何实现的，并从头构建一个LSTM。

### 从头构建LSTM

为了构建一个LSTM，我们首先需要理解它的架构。<<lstm>>展示了它的内部结构。

<img src="images/LSTM.png" id="lstm" caption="LSTM的架构" alt="一个图表展示了LSTM的内部架构" width="700">

在这张图中，我们的输入$x_{t}$与前一个隐藏状态（$h_{t-1}$）和细胞状态（$c_{t-1}$）一起从左侧进入。四个橙色框代表四个层（我们的神经网络），激活函数是sigmoid（$\sigma$）或tanh。tanh只是一个sigmoid函数，重新调整到-1到1的范围内。它的数学表达式可以写成这样：

$$\tanh(x) = \frac{e^{x} - e^{-x}}{e^{x}+e^{-x}} = 2 \sigma(2x) - 1$$

其中$\sigma$是sigmoid函数。绿色圆圈是逐元素操作。右侧输出的是新的隐藏状态（$h_{t}$）和新的细胞状态（$c_{t}$），准备接收下一个输入。新的隐藏状态也被用作输出，这就是为什么箭头分叉向上。

让我们逐一解释这四个神经网络（称为*门*），但在这之前，注意细胞状态（顶部）几乎没有变化。它甚至没有直接通过神经网络！这正是它能够保持更长期状态的原因。

首先，输入和旧隐藏状态的箭头连接在一起。在我们本章早些时候写的RNN中，我们将它们相加。在LSTM中，我们将它们堆叠在一个大张量中。这意味着我们的嵌入维度（即$x_{t}$的维度）可以与我们的隐藏状态维度不同。如果我们称这些为`n_in`和`n_hid`，底部的箭头大小为`n_in + n_hid`；因此所有神经网络（橙色框）都是具有`n_in + n_hid`输入和`n_hid`输出的线性层。

从左到右看，第一个门称为*遗忘门*。由于它是一个线性层后跟sigmoid，它的输出将由0到1之间的标量组成。我们将这个结果乘以细胞状态，以决定保留哪些信息，丢弃哪些信息：接近0的值被丢弃，接近1的值被保留。这给了LSTM忘记其长期状态的能力。例如，当遇到一个句号或`xxbos`标记时，我们希望它（已经学会）重置其细胞状态。

第二个门称为*输入门*。它与第三个门（实际上没有名字，但有时被称为*细胞门*）一起更新细胞状态。例如，我们可能看到一个新的性别代词，在这种情况下，我们需要替换遗忘门移除的性别信息。与遗忘门类似，输入门决定更新细胞状态的哪些元素（接近1的值）或不更新（接近0的值）。第三个门决定这些更新值是什么，在-1到1的范围内（感谢tanh函数）。结果然后被加到细胞状态上。

最后一个门是*输出门*。它决定从细胞状态中使用哪些信息来生成输出。细胞状态在与输出门的sigmoid输出结合之前先通过tanh，结果是新的隐藏状态。

在代码方面，我们可以这样写这些步骤：

In [ ]:
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.forget_gate = nn.Linear(ni + nh, nh)
        self.input_gate  = nn.Linear(ni + nh, nh)
        self.cell_gate   = nn.Linear(ni + nh, nh)
        self.output_gate = nn.Linear(ni + nh, nh)

    def forward(self, input, state):
        h,c = state
        h = torch.cat([h, input], dim=1)
        forget = torch.sigmoid(self.forget_gate(h))
        c = c * forget
        inp = torch.sigmoid(self.input_gate(h))
        cell = torch.tanh(self.cell_gate(h))
        c = c + inp * cell
        out = torch.sigmoid(self.output_gate(h))
        h = out * torch.tanh(c)
        return h, (h,c)

在实践中，我们可以重构代码。此外，在性能方面，执行一次大的矩阵乘法比执行四次小的矩阵乘法更好（这是因为我们只在GPU上启动一次特殊的快速内核，它可以让GPU并行处理更多的工作）。堆叠需要一些时间（因为我们必须在GPU上移动一个张量，以便将其全部放在一个连续的数组中），所以我们为输入和隐藏状态使用两个单独的层。优化和重构后的代码看起来像这样：

In [ ]:
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.ih = nn.Linear(ni,4*nh)
        self.hh = nn.Linear(nh,4*nh)

    def forward(self, input, state):
        h,c = state
        # One big multiplication for all the gates is better than 4 smaller ones
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)
        ingate,forgetgate,outgate = map(torch.sigmoid, gates[:3])
        cellgate = gates[3].tanh()

        c = (forgetgate*c) + (ingate*cellgate)
        h = outgate * c.tanh()
        return h, (h,c)

在这里，我们使用PyTorch的`chunk`方法将我们的张量分成四部分。它的工作原理如下：

In [ ]:
t = torch.arange(0,10); t

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
t.chunk(2)

(tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]))

现在让我们使用这个架构来训练一个语言模型！

### Training a Language Model Using LSTMs

这是与`LMModel5`相同的网络，使用两层LSTM。我们可以以更高的学习率训练它，训练时间更短，并获得更好的准确率：

In [ ]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.000821,2.663942,0.438314,00:02
1,2.139642,2.184780,0.240479,00:02
2,1.607275,1.812682,0.439779,00:02
3,1.347711,1.830982,0.497477,00:02
4,1.123113,1.937766,0.594401,00:02
5,0.852042,2.012127,0.631592,00:02
6,0.565494,1.312742,0.725749,00:02
7,0.347445,1.297934,0.711263,00:02
8,0.208191,1.441269,0.731201,00:02
9,0.126335,1.569952,0.737305,00:02


现在这比多层RNN好多了！然而，我们仍然可以看到有一点过拟合，这表明一点正则化可能会有所帮助。

## 正则化 LSTM

循环神经网络（RNN）通常难以训练，因为之前我们看到的激活值和梯度消失问题。使用LSTM（或GRU）单元比使用普通RNN更容易训练，但它们仍然非常容易过拟合。数据增强虽然是一种可能性，但对于文本数据来说，它的使用不如图像数据那么普遍，因为在大多数情况下，它需要另一个模型来生成随机增强（例如，通过将文本翻译成另一种语言，然后再翻译回原始语言）。总的来说，文本数据的数据增强目前还没有得到很好的探索。

然而，我们可以使用其他正则化技术来减少过拟合，这些技术在Stephen Merity、Nitish Shirish Keskar和Richard Socher的论文["Regularizing and Optimizing LSTM Language Models"](https://arxiv.org/abs/1708.02182)中已经得到了彻底的研究。这篇论文展示了如何有效地使用*dropout*、*激活正则化*和*时间激活正则化*可以使LSTM超越之前需要更复杂模型才能达到的最新成果。作者将使用这些技术的LSTM称为*AWD-LSTM*。我们将依次看看这些技术。

### Dropout

Dropout是由Geoffrey Hinton等人在[Improving neural networks by preventing co-adaptation of feature detectors](https://arxiv.org/abs/1207.0580)中引入的一种正则化技术。基本思想是在训练时随机地将一些激活值改为零。这确保了所有神经元都在积极地为输出工作，如<<img_dropout>>所示（来自Nitish Srivastava等人的"Dropout: A Simple Way to Prevent Neural Networks from Overfitting"）。

<img src="images/Dropout1.png" alt="展示神经网络中dropout的图示" width="800" id="img_dropout" caption="在神经网络中应用dropout（Nitish Srivastava等人提供）">

Hinton在一次采访中用一个有趣的比喻解释了dropout的灵感来源：

> : 我去了银行。出纳员们一直在换，我问其中一个为什么。他说他不知道，但他们经常变动。我想这一定是因为需要员工之间的合作才能成功欺诈银行。这让我意识到，如果在每个例子中随机移除不同的神经元子集，就能防止串谋，从而减少过拟合。

在同一个采访中，他还解释了神经科学提供的额外灵感：

> : 我们真的不知道神经元为什么会尖峰放电。一种理论是它们想要保持噪声，以便正则化，因为我们的参数比数据点多得多。dropout的想法是，如果你有噪声激活，你就可以使用一个更大的模型。

这解释了为什么dropout有助于泛化的背后理念：首先，它帮助神经元更好地相互协作，然后它使激活值变得更嘈杂，从而使模型更加稳健。

然而，我们可以看到，如果我们只是将这些激活值置零而不做其他处理，我们的模型在训练时会遇到问题：如果我们从五个激活值的总和（由于我们应用了ReLU，它们都是正数）变为只有两个，这将不会具有相同的规模。因此，如果我们以概率`p`应用dropout，我们会重新调整所有激活值，通过将它们除以`1-p`（平均来说`p`将被置零，所以剩下`1-p`），如<<img_dropout1>>所示。

<img src="images/Dropout.png" alt="介绍dropout的文章中的图示，展示了一个神经元的开/关状态" width="600" id="img_dropout1" caption="为什么在应用dropout时要缩放激活值（Nitish Srivastava等人提供）">

这是PyTorch中dropout层的完整实现（尽管PyTorch的原生层实际上是用C语言编写的，而不是Python）：

In [ ]:
class Dropout(Module):
    def __init__(self, p): self.p = p
    def forward(self, x):
        if not self.training: return x
        mask = x.new(*x.shape).bernoulli_(1-p)
        return x * mask.div_(1-p)

`bernoulli_`方法创建了一个随机零（概率为`p`）和一（概率为`1-p`）的张量，然后将其与我们的输入相乘，再除以`1-p`。注意使用`training`属性，这是任何PyTorch `nn.Module`中都有的，它告诉我们我们是在训练还是推理。

> 注意：自己做实验：在本书的前几章中，我们会在这里添加一个`bernoulli_`的代码示例，以便你可以看到它的确切工作方式。但现在你已经足够了解如何自己做这件事，我们将越来越少地为你做示例，而是期望你自己做实验来了解我们正在学习的代码是如何工作的。在这种情况下，你会在本章末的问题中看到我们要求你实验`bernoulli_`——但不要等到我们要求你实验才去发展你对代码的理解；无论如何都要去做！

在使用dropout之前，将我们的LSTM输出传递给最终层将有助于减少过拟合。Dropout也用于许多其他模型中，包括`fastai.vision`中使用的默认CNN头部，并且在`fastai.tabular`中通过传递`ps`参数也可用（其中每个"p"都传递给每个添加的`Dropout`层），正如我们将在<<chapter_arch_details>>中看到的。

Dropout在训练和验证模式下有不同的行为，我们使用`Dropout`中的`training`属性来指定这一点。在`Module`上调用`train`方法将`training`设置为`True`（对于你调用该方法的模块以及它递归包含的每个模块都是如此），而`eval`将其设置为`False`。当调用`Learner`的方法时，这会自动完成，但如果你不使用该类，请记得根据需要在两者之间切换。

### 激活正则化和时间激活正则化

*激活正则化*（AR）和*时间激活正则化*（TAR）是两种与<<chapter_collab>>中讨论的权重衰减非常相似的正则化方法。在应用权重衰减时，我们会在损失函数中添加一个小的惩罚项，目的是使权重尽可能小。对于激活正则化，我们将尝试使LSTM产生的最终激活值尽可能小，而不是权重。

为了正则化最终激活值，我们必须在某个地方存储它们，然后在损失函数中添加它们的平方和的平均值（以及一个乘数`alpha`，它就像权重衰减的`wd`）：

``` python
loss += alpha * activations.pow(2).mean()
```

这种方法鼓励模型产生较小的激活值，从而有助于防止过拟合。在实际应用中，`alpha`的值需要通过实验来确定，以便在正则化效果和模型性能之间找到合适的平衡。

时间激活正则化（TAR）与我们在句子中预测标记的事实有关。这意味着当我们按顺序阅读我们的LSTM输出时，它们应该在某种程度上是有意义的。TAR通过在损失函数中添加一个惩罚项来鼓励这种行为，使得两个连续激活之间的差异尽可能小：我们的激活张量的形状是`bs x sl x n_hid`，我们在序列长度轴（中间的维度）上读取连续的激活。这样，TAR可以表示为：

``` python
loss += beta * (activations[:,1:] - activations[:,:-1]).pow(2).mean()
```

`alpha`和`beta`是两个需要调整的超参数。为了使这工作，我们需要我们的模型（带有dropout）返回三样东西：正确的输出、dropout之前的LSTM激活值，以及dropout之后的LSTM激活值。AR通常应用于dropout后的激活（以免惩罚我们之后将其置零的激活），而TAR则应用于未dropout的激活（因为这些零会在两个连续时间步之间产生大的差异）。然后有一个名为`RNNRegularizer`的回调，它将为我们应用这种正则化。

### 训练一个权重绑定的正则化LSTM

我们可以结合dropout（在我们进入输出层之前应用）与AR和TAR来训练我们之前的LSTM。我们只需要返回三样东西而不是一样：我们LSTM的正常输出、dropout后的激活值，以及我们LSTM的激活值。后两者将被回调`RNNRegularization`捕获，用于对损失的贡献。

我们可以从[AWD LSTM论文](https://arxiv.org/abs/1708.02182)中添加另一个有用的技巧是*权重绑定*。在语言模型中，输入嵌入表示从英语单词到激活的映射，输出隐藏层表示从激活到英语单词的映射。我们可能直观地期望，这些映射可能是相同的。我们可以通过将相同的权重矩阵分配给这些层来在PyTorch中表示这一点：

```python
self.h_o.weight = self.i_h.weight
```

在`LMModel7`中，我们包含了这些最后的调整：

In [ ]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

我们可以创建一个使用`RNNRegularizer`回调的正则化`Learner`：

In [ ]:
learn = Learner(dls, LMModel7(len(vocab), 64, 2, 0.5),
                loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                cbs=[ModelResetter, RNNRegularizer(alpha=2, beta=1)])

`TextLearner`会自动为我们添加这两个回调（默认情况下使用这些值的`alpha`和`beta`），因此我们可以将前面的行简化为：

In [ ]:
learn = TextLearner(dls, LMModel7(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

然后我们可以训练模型，并通过将权重衰减增加到`0.1`来添加额外的正则化：

In [ ]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,2.693885,2.013484,0.466634,00:02
1,1.685549,1.187310,0.629313,00:02
2,0.973307,0.791398,0.745605,00:02
3,0.555823,0.640412,0.794108,00:02
4,0.351802,0.557247,0.836100,00:02
5,0.244986,0.594977,0.807292,00:02
6,0.192231,0.511690,0.846761,00:02
7,0.162456,0.520370,0.858073,00:02
8,0.142664,0.525918,0.842285,00:02
9,0.128493,0.495029,0.858073,00:02


现在这个模型比我们之前的模型好多了！

## 总结

你现在已经看到了我们在<<chapter_nlp>>中用于文本分类的AWD-LSTM架构中的所有内容。它在更多地方使用了dropout：

- 嵌入dropout（在嵌入层内部，随机丢弃一些嵌入行）
- 输入dropout（在嵌入层之后应用）
- 权重dropout（在每个训练步骤中应用于LSTM的权重）
- 隐藏dropout（应用于两层之间的隐藏状态）

这使得它更加正则化。由于微调这五个dropout值（包括输出层之前的dropout）是复杂的，我们已经确定了一些好的默认值，并允许通过你在该章节中看到的`drop_mult`参数整体调整dropout的幅度（该参数会乘以每个dropout）。

另一种非常强大的架构，特别是在“序列到序列”问题中（即，因变量本身是一个可变长度序列的问题，如语言翻译），是Transformers架构。你可以在[书的网站](https://book.fast.ai/)的奖励章节中找到它。

## 问卷调查

1. 如果你的项目数据集非常大且复杂，以至于处理它需要花费大量时间，你应该做什么？
2. 为什么在创建语言模型之前我们要将数据集中的文档连接起来？
3. 为了使用标准的全连接网络根据前三个词预测第四个词，我们需要对模型进行哪两个调整？
4. 我们如何在PyTorch中在多个层之间共享权重矩阵？
5. 编写一个模块，根据句子的前两个词预测第三个词，不要偷看。
6. 什么是循环神经网络？
7. “隐藏状态”是什么？
8. 在`LMModel1`中，隐藏状态的等价物是什么？
9. 为了在RNN中保持状态，为什么按顺序传递文本给模型很重要？
10. RNN的“展开”表示是什么？
11. 为什么在RNN中保持隐藏状态会导致内存和性能问题？我们如何修复这个问题？
12. 什么是“BPTT”？
13. 编写代码以打印验证集的前几个批次，包括将标记ID转换回英文字符串，就像我们在<<chapter_nlp>>中对IMDb数据批次展示的那样。
14. `ModelResetter`回调是什么？为什么我们需要它？
15. 仅预测每三个输入词的一个输出词有什么缺点？
16. 为什么我们需要为`LMModel4`自定义损失函数？
17. 为什么`LMModel4`的训练不稳定？
18. 在展开表示中，我们可以看到循环神经网络实际上有很多层。那么我们为什么需要堆叠RNN来获得更好的结果？
19. 绘制一个堆叠（多层）RNN的表示图。
20. 如果我们在RNN中不那么频繁地调用`detach`，为什么我们可能会获得更好的结果？为什么在实践中，对于一个简单的RNN，这可能不会发生？
21. 为什么深度网络会导致非常大或非常小的激活值？为什么这很重要？
22. 在计算机的浮点数表示中，哪些数字最精确？
23. 为什么梯度消失会阻止训练？
24. 在LSTM架构中，为什么有两个隐藏状态是有帮助的？每个的目的是什么？
25. 在LSTM中，这两个状态叫什么？
26. tanh是什么，它与sigmoid有什么关系？
27. `LSTMCell`中的这段代码的目的是什么：`h = torch.cat([h, input], dim=1)`？
28. PyTorch中的`chunk`是做什么的？
29. 仔细研究`LSTMCell`的重构版本，确保你理解它是如何和未重构版本做同样的事情的。
30. 为什么我们可以为`LMModel6`使用更高的学习率？
31. AWD-LSTM模型中使用了哪三种正则化技术？
32. “dropout”是什么？
33. 为什么我们要在dropout中缩放激活值？这是在训练期间应用的，还是在推理期间，或者两者都应用？
34. `Dropout`中的这行代码的目的是什么：`if not self.training: return x`？
35. 尝试使用`bernoulli_`来理解它的工作原理。
36. 在PyTorch中，如何将你的模型设置为训练模式？在评估模式下呢？
37. 写出激活正则化的方程（以数学或代码形式，随你喜欢）。它与权重衰减有什么不同？
38. 写出时间激活正则化的方程（以数学或代码形式，随你喜欢）。为什么我们不会在计算机视觉问题中使用这个？
39. 在语言模型中，“权重绑定”是什么？

### 进一步研究

1. 在`LMModel2`中，为什么`forward`可以以`h=0`开始？为什么我们不需要说`h=torch.zeros(...)`？
2. 从头开始编写一个LSTM的代码（你可以参照<<lstm>>）。
3. 在互联网上搜索GRU架构，并从头开始实现它，尝试训练一个模型。看看你能否得到与本章中看到的类似结果。将你的结果与PyTorch内置的`GRU`模块的结果进行比较。
4. 查看fastai中AWD-LSTM的源代码，并尝试将每一行代码映射到本章展示的概念上。